In [2]:
import os
import shutil
import pickle # We will use pickle to save our list of chunks
from tqdm import tqdm
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

print("--- PHASE 1: Loading and Chunking Documents ---")

# -------------- STEP 1: FIND ALL PDF FILES --------------
downloaded_folder = "../Database/arxiv_data"
all_pdf_paths = []
print(f"📂 Searching for PDF files in '{downloaded_folder}'...")
for root, dirs, files in os.walk(downloaded_folder):
    for file in files:
        if file.endswith(".pdf"):
            all_pdf_paths.append(os.path.join(root, file))
print(f"👍 Found {len(all_pdf_paths)} PDF files.")

# -------------- STEP 2: LOAD PDFS --------------
print("\nLoading PDF content...")
all_docs = []
for path in tqdm(all_pdf_paths, desc="Loading PDFs"):
    try:
        loader = PyPDFLoader(path)
        docs = loader.load()
        filename = os.path.basename(path).replace(".pdf", "")
        for doc in docs:
            doc.metadata["source"] = f"https://arxiv.org/pdf/{filename}"
        all_docs.extend(docs)
    except Exception as e:
        print(f"⚠️ Warning: Failed to load {path}: {e}")

# -------------- STEP 3: SANITIZE AND SPLIT --------------
print("\nSanitizing and splitting documents...")
sanitized_docs = []
for doc in all_docs:
    if isinstance(doc.page_content, str) and doc.page_content.strip():
        sanitized_docs.append(doc)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
chunks = text_splitter.split_documents(sanitized_docs)
print(f"Split documents into {len(chunks)} chunks.")

# -------------- STEP 4: FINAL CLEANING AND SAVING --------------
print("\nPerforming final cleaning and saving chunks to a file...")
final_chunks = []
for chunk in tqdm(chunks, desc="Cleaning Chunks"):
    if isinstance(chunk.page_content, str) and chunk.page_content.strip():
        final_chunks.append(chunk)

# Save the clean chunks to a file using pickle
output_path = "../Database/processed_chunks.pkl"
with open(output_path, "wb") as f:
    pickle.dump(final_chunks, f)

print(f"\n✅ PHASE 1 COMPLETE! Cleaned {len(final_chunks)} chunks and saved them to '{output_path}'.")

--- PHASE 1: Loading and Chunking Documents ---
📂 Searching for PDF files in '../Database/arxiv_data'...
👍 Found 865 PDF files.

Loading PDF content...


Loading PDFs:   1%|▎                            | 8/865 [00:03<03:37,  3.93it/s]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 60 0 (offset 0)
Ignoring wrong pointing object 62 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing objec

⚠️ Warning: Failed to load ../Database/arxiv_data/general_literature/2005.04681v2_Simulating_quantum_dynamics:_Evolution_of_algorith.pdf: Stream has ended unexpectedly


Loading PDFs:  31%|████████▍                  | 271/865 [01:45<02:57,  3.34it/s]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong pointing object 93 0 (offset 0)
Ignoring wrong pointing object 95 0 (offset 0)
Ignoring wrong pointing object 117 0 (offset 0)
Ignoring wrong pointing object 119 0 (offset 0)
Loading PDFs:  32%|████████

⚠️ Warning: Failed to load ../Database/arxiv_data/artificial_intelligence/2412.01992v1_ChatCollab:_Exploring_Collaboration_Between_Humans.pdf: Stream has ended unexpectedly


...

⚠️ Warning: Failed to load ../Database/arxiv_data/artificial_intelligence/2204.10358v1_Creative_Problem_Solving_in_Artificially_Intellige.pdf: Stream has ended unexpectedly


Loading PDFs:  39%|██████████▍                | 334/865 [02:01<01:51,  4.75it/s]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Loading PDFs:  40%|██████████▉                | 349/865 [02:09<07:05,  1.21it/s]could not convert string to float: b'0.00-24254184' : FloatObject (b'0.00-24254184') invalid; use 0.0 instead
could not convert string to float: b'0.00-91074704' : FloatObject (b'0.00-91074704') invalid; use 0.0 instead
could not convert string to float: b'0.00-19984013' : FloatObject (b'0.00-19984013') invalid; use 0.0 ins


Sanitizing and splitting documents...
Split documents into 71900 chunks.

Performing final cleaning and saving chunks to a file...


Cleaning Chunks: 100%|███████████████| 71900/71900 [00:00<00:00, 2343933.29it/s]



✅ PHASE 1 COMPLETE! Cleaned 71900 chunks and saved them to '../Database/processed_chunks.pkl'.


In [1]:
import os
import shutil
import pickle
from tqdm import tqdm
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

print("--- PHASE 2: Ingesting Chunks into ChromaDB (Final Robust Method) ---")

# --- Configuration ---
CHUNK_FILE_PATH = "../Database/processed_chunks.pkl"
PERSIST_DIRECTORY = "../Database/Vector-DB-New"
BATCH_SIZE = 500

# --- Setup ---
if os.path.exists(PERSIST_DIRECTORY):
    print(f"🧹 Deleting existing database at '{PERSIST_DIRECTORY}'.")
    shutil.rmtree(PERSIST_DIRECTORY)

print(f"Loading processed chunks from '{CHUNK_FILE_PATH}'...")
with open(CHUNK_FILE_PATH, "rb") as f:
    final_chunks = pickle.load(f)
print(f"👍 Loaded {len(final_chunks)} chunks.")

embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'mps'}
)

# --- Create the DB with the FIRST batch ---
print("\nCreating the initial database with the first batch...")
first_batch = final_chunks[:BATCH_SIZE]
remaining_chunks = final_chunks[BATCH_SIZE:]

first_batch_texts = [chunk.page_content for chunk in first_batch]
first_batch_metadatas = [chunk.metadata for chunk in first_batch]
for metadata in first_batch_metadatas:
    for key, value in metadata.items():
        if not isinstance(value, (str, int, float, bool)):
            metadata[key] = str(value)

# This creates and saves the initial DB.
Chroma.from_texts(
    texts=first_batch_texts,
    metadatas=first_batch_metadatas,
    embedding=embeddings_model,
    persist_directory=PERSIST_DIRECTORY
)
print(f"✅ Initial database created with the first {len(first_batch)} chunks.")

# 💡 --- CRITICAL FIX: Re-load the database to get a live, writable connection --- 💡
print("Re-loading database to ensure a writable connection...")
vector_db = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings_model
)
print("✅ Connection re-established.")

# --- Ingest the REST of the batches ---
print(f"\nStarting ingestion for the remaining {len(remaining_chunks)} chunks...")
for i in tqdm(range(0, len(remaining_chunks), BATCH_SIZE), desc="Ingesting Batches"):
    batch = remaining_chunks[i:i + BATCH_SIZE]
    
    batch_texts = [chunk.page_content for chunk in batch]
    batch_metadatas = [chunk.metadata for chunk in batch]
    
    for metadata in batch_metadatas:
        for key, value in metadata.items():
            if not isinstance(value, (str, int, float, bool)):
                metadata[key] = str(value)
    try:
        vector_db.add_texts(texts=batch_texts, metadatas=batch_metadatas)
    except Exception as e:
        batch_start_index = BATCH_SIZE + i
        print(f"⚠️ ERROR in batch starting at index {batch_start_index}: {e}")
        
# Persist any final changes
print("\nPersisting final database changes...")
vector_db.persist()

print(f"\n🚀 PHASE 2 COMPLETE! Database saved at '{PERSIST_DIRECTORY}'.")

--- PHASE 2: Ingesting Chunks into ChromaDB (Final Robust Method) ---
Loading processed chunks from '../Database/processed_chunks.pkl'...
👍 Loaded 71900 chunks.


/var/folders/5n/8m4t00412h97v592vh5y0pgc0000gn/T/ipykernel_91435/641433560.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(



Creating the initial database with the first batch...


/var/folders/5n/8m4t00412h97v592vh5y0pgc0000gn/T/ipykernel_91435/641433560.py:53: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(


✅ Initial database created with the first 500 chunks.
Re-loading database to ensure a writable connection...
✅ Connection re-established.

Starting ingestion for the remaining 71400 chunks...


Ingesting Batches:  88%|███████████████████▍  | 126/143 [07:10<00:44,  2.64s/it]

⚠️ ERROR in batch starting at index 63000: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]


Ingesting Batches: 100%|██████████████████████| 143/143 [08:11<00:00,  3.44s/it]


Persisting final database changes...

🚀 PHASE 2 COMPLETE! Database saved at '../Database/Vector-DB-New'.



/var/folders/5n/8m4t00412h97v592vh5y0pgc0000gn/T/ipykernel_91435/641433560.py:79: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()
